In [1]:
# installing some libraries
!pip3 install textstat
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip3 install nltk==3.6.5

     |████████████████████████████████| 13.9 MB 5.2 MB/s            


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import split, explode, udf, lit, size, col, isnan
from pyspark.sql.types import IntegerType, ArrayType, DoubleType, FloatType
from collections import Counter
from scipy import stats
import spacy
from textstat.textstat import textstatistics, legacy_round
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# show all columns in pandas
pd.set_option('display.max_columns', None)

## Create Spark Session

In [4]:
# create spark session
spark = SparkSession.builder\
            .appName('eecs-e6893-project')\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Load Dataset

In [5]:
# specifying global variables
BUCKET_NAME = "eecs-e6893-project-42"
DATA_FILE = "gs://eecs-e6893-project-42/dataset/data_final.csv"
STOP_WORDS = stopwords.words('english')

In [18]:
# loading the dataframe using pyspark
df = spark.read.csv(DATA_FILE, header=True, inferSchema=True, multiLine=True, mode="DROPMALFORMED", escape=",", 
                    quote="\"")

In [19]:
pd.DataFrame(df.take(100), columns=df.columns)

,_c0,Id,AcceptedAnswerId,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,Reputation,Views,UpVotes,DownVotes,Quality,Text
0,0,8019330,8019355.0,-1,246.0,<p>My lines:</p>\n\n<pre><code>ABC: XYZ\nXYZ A...,935420,"In Perl, how can I join an input line with the...",<perl>,6.0,4,None,103,31,0,0,0,my lines:\nabc: xyz\nxyz abc: xyz\nxyz:\nabc\n...
1,5,13579810,13579922.0,54,259088.0,<p>I have a 350MB file named <code>text_file.t...,1342109,How to import data from text file to mysql dat...,<mysql><import><text-files>,8.0,0,32.0,19464,1669,83,1,1,i have a 350mb file named text_file.txt contai...
2,8,3934106,3934115.0,10,3153.0,<p>This is a curiosity more than anything: Doe...,348056,Unicode based programming language,<unicode><programming-languages>,13.0,10,5.0,20747,1040,461,17,1,this is a curiosity more than anything: does t...
3,9,4724606,4724676.0,60,56061.0,<p>I need to check (in Javascript) if a CSS fi...,244415,How to use Javascript to check and load CSS if...,<javascript><jquery><css>,14.0,0,19.0,3631,80,573,1,1,i need to check (in javascript) if a css file ...
4,12,28980152,31768957.0,12,17287.0,<p>Is there a way to play sound out of the App...,2158235,Play sound from Apple Watch speaker,<ios><objective-c><iphone><audio><apple-watch>,6.0,6,8.0,379,46,40,1,1,is there a way to play sound out of the apple ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,340,1553314,1553390.0,12,18568.0,<p>How can I fetch a stock's price from the Au...,177913,How to fetch stock price from Australia Stock ...,<corporate><stockquotes>,10.0,0,4.0,355,60,10,0,1,how can i fetch a stock's price from the austr...
96,341,1637581,1638813.0,49,45169.0,<p>My colleagues are using Visual Studio 2002 ...,198145,C++ MFC vs .NET?,<c#><.net><c++><mfc><visual-c++>,11.0,4,17.0,5566,978,677,20,1,my colleagues are using visual studio 2002 and...
97,342,22613229,22613388.0,1,145.0,<p>What is difference between this</p>\n\n<pre...,3455952,About instantiate a class from inherited class,<c#><class><inheritance><polymorphism>,7.0,2,None,21,1,0,0,1,what is difference between this\nderivedclass ...
98,344,3499572,3500534.0,13,9618.0,<p>What is the default access modifier of a cl...,416941,What is the default access modifier of a class?,<.net><asp.net>,7.0,0,3.0,1140,289,9,1,1,what is the default access modifier of a class?\n


In [11]:
df = df.drop('_c0')
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- AcceptedAnswerId: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- ViewCount: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: string (nullable = true)
 |-- CommentCount: string (nullable = true)
 |-- FavoriteCount: string (nullable = true)
 |-- Reputation: string (nullable = true)
 |-- Views: string (nullable = true)
 |-- UpVotes: string (nullable = true)
 |-- DownVotes: string (nullable = true)
 |-- Quality: string (nullable = true)
 |-- Text: string (nullable = true)



In [20]:
print((df.count(), len(df.columns)))

(2190127, 18)


## Data Cleaning

- Segregate code and body
- remove html tags
- remove records with malformed ids, quality

In [34]:
@udf
def get_code(text):
    if text is None:
        return ""
    pattern = r"<code.*?>([\s\S]*?)<\/code>"
    processed_text = re.findall(pattern, text)
    return "".join(processed_text)

@udf
def get_body(text):
    if text is None:
        return ""
    pattern = r"<code.*?>([\s\S]*?)<\/code>"
    matched_obj = re.finditer(pattern, text, re.MULTILINE)
    output = list()
    for obj in matched_obj:
        pattern_temp = obj[0]
        text = text.replace(pattern_temp, '')
    pattern = r"<[^>]*>"
    processed_text = re.sub(pattern, "", text)
    return processed_text

In [38]:
clean_df = df.where(isnan(col('Body')) == False)
clean_df = clean_df.filter(col("Id").rlike(r"^([0-9]+)$"))
clean_df = clean_df.withColumn("Score", clean_df["Score"].cast(IntegerType()))
clean_df = clean_df.withColumn("ViewCount", clean_df["ViewCount"].cast(IntegerType()))
clean_df = clean_df.withColumn("AnswerCount", clean_df["AnswerCount"].cast(IntegerType()))
clean_df = clean_df.withColumn("CommentCount", clean_df["CommentCount"].cast(IntegerType()))
clean_df = clean_df.withColumn("FavoriteCount", clean_df["FavoriteCount"].cast(IntegerType()))
clean_df = clean_df.withColumn("Reputation", clean_df["Reputation"].cast(IntegerType()))
clean_df = clean_df.withColumn("Views", clean_df["Views"].cast(IntegerType()))
clean_df = clean_df.withColumn("UpVotes", clean_df["UpVotes"].cast(IntegerType()))
clean_df = clean_df.withColumn("DownVotes", clean_df["DownVotes"].cast(IntegerType()))
clean_df = clean_df.withColumn("Quality", clean_df["Quality"].cast(IntegerType()))
clean_df = clean_df.withColumn("CodeBody", get_code(df['Body']))
clean_df = clean_df.withColumn("CleanBody", get_body(df['Body']))
clean_df = clean_df.fillna(0)

In [36]:
clean_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- AcceptedAnswerId: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- Quality: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- CodeBody: string (nullable = true)
 |-- CleanBody: string (nullable = true)



In [39]:
pd.DataFrame(clean_df.take(50), columns=clean_df.columns)

,_c0,Id,AcceptedAnswerId,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,Reputation,Views,UpVotes,DownVotes,Quality,Text,CodeBody,CleanBody
0,0,8019330,8019355.0,-1,246,<p>My lines:</p>\n\n<pre><code>ABC: XYZ\nXYZ A...,935420,"In Perl, how can I join an input line with the...",<perl>,6,4,0,103,31,0,0,0,my lines:\nabc: xyz\nxyz abc: xyz\nxyz:\nabc\n...,ABC: XYZ\nXYZ ABC: XYZ\nXYZ:\nABC\nXYZ\nABC:\n...,My lines:\n\n\n\nI want to join lines end with...
1,5,13579810,13579922.0,54,259088,<p>I have a 350MB file named <code>text_file.t...,1342109,How to import data from text file to mysql dat...,<mysql><import><text-files>,8,0,32,19464,1669,83,1,1,i have a 350mb file named text_file.txt contai...,text_file.txt345868230 1646198120 153128314...,I have a 350MB file named containing this tab...
2,8,3934106,3934115.0,10,3153,<p>This is a curiosity more than anything: Doe...,348056,Unicode based programming language,<unicode><programming-languages>,13,10,5,20747,1040,461,17,1,this is a curiosity more than anything: does t...,,This is a curiosity more than anything: Does t...
3,9,4724606,4724676.0,60,56061,<p>I need to check (in Javascript) if a CSS fi...,244415,How to use Javascript to check and load CSS if...,<javascript><jquery><css>,14,0,19,3631,80,573,1,1,i need to check (in javascript) if a css file ...,,I need to check (in Javascript) if a CSS file ...
4,12,28980152,31768957.0,12,17287,<p>Is there a way to play sound out of the App...,2158235,Play sound from Apple Watch speaker,<ios><objective-c><iphone><audio><apple-watch>,6,6,8,379,46,40,1,1,is there a way to play sound out of the apple ...,,Is there a way to play sound out of the Apple ...
5,15,307688,None,376,507072,<p>What is a simple way of downloading a file ...,20837,How to download a file from a URL in C#?,<c#><downloadfile>,11,1,70,4970,226,5,1,1,what is a simple way of downloading a file fro...,,What is a simple way of downloading a file fro...
6,20,913671,914383.0,32,25655,<p>Are there native code compilers for Lisp? T...,45269,Are there Lisp native code compilers?,<compiler-construction><lisp>,6,4,30,5625,411,55,3,1,are there native code compilers for lisp? to w...,,Are there native code compilers for Lisp? To w...
7,24,1203892,1205824.0,215,174146,<p>I need to know the current route in a filte...,120496,How can I find out the current route in Rails?,<ruby-on-rails><ruby><routing>,13,2,69,10691,426,86,9,1,i need to know the current route in a filter i...,,I need to know the current route in a filter i...
8,27,15434728,15434783.0,27,72739,"<p>I have a file containing many lines, and I ...",1003575,How to display the first word of each line in ...,<linux><shell><grep>,6,3,11,34364,4157,1079,35,1,"i have a file containing many lines, and i wan...",,"I have a file containing many lines, and I wan..."
9,28,6347039,6347078.0,9,17203,"<p>I'm developing a web application, using ASP...",586528,How to process payments through cards?,<c#><asp.net><e-commerce><payment-gateway><cre...,6,0,6,17348,418,165,2,0,"i'm developing a web application, using asp.ne...",,"I'm developing a web application, using ASP.ne..."


In [40]:
print((clean_df.count(), len(clean_df.columns)))

(79246, 20)


## Feature Extraction

In [25]:
# helper functions for various features
def fetch_sentences(text):
    output = list()
    sentence = ""
    for c in text:
        sentence += c
        if c == '.' or c == '!' or c == '?':
            output.append(sentence)
            sentence = ""
            continue
    return output

def word_count(text):
    sentences = fetch_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words

def sentence_count(text):
    return len(fetch_sentences(text))
 
def avg_sentence_length(text):
    return float(word_count(text) / sentence_count(text))

def character_count(text):
    return sum(len(word) for word in text.split(" "))

def syllables_count(word):
    return textstatistics().syllable_count(word)

def syllables_count_temp(word):
    word = word.lower()
    return len(
        re.findall('(?!e$)[aeiouy]+', word, re.I) +
        re.findall('^[^aeiouy]*e$', word, re.I)
    )

def total_syllables_count(text):
    return sum(syllables_count_temp(word) for word in text.split(" "))

def avg_syllables_per_word(text):
    return legacy_round(float(syllables_count_temp(text)) / float(word_count(text), 1))

def poly_syllable_count(text):
    count = 0
    words = []
    sentences = fetch_sentences(text)
    for sentence in sentences:
        words += [token for token in sentence]
    for word in words:
        syllable_count = syllables_count_temp(str(word))
        if syllable_count >= 3:
            count += 1
    return count

def difficult_words(text):
    # Find all words in the text
    words = []
    sentences = fetch_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    # difficult words are those with syllables >= 2
    # easy_word_set is provide by Textstat as
    # a list of common words
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count_temp(word)
        if word not in STOP_WORDS and syllable_count >= 3:
            diff_words_set.add(word)
 
    return len(diff_words_set)

def avgLetters(text):
    #returns average number of letters per 100 words in a text file
    #uses a list of 100 word chunks to calculate this

    word_list = text.split()
    intervals = range(0, len(word_list), 100)
    word_chunks = [word_list[n:n+100] for n in intervals]
    lettersList = []
    for n in range(0, len(intervals)):
        words = [len(i) for i in word_chunks[n]]
        letters = sum(words)
        lettersList.extend([letters])
    L = sum(lettersList)
    if len(word_chunks) != 0:
        return float(L/len(word_chunks))
    return 0.0

def avgSentences(text):
    #takes entire text and returns average number of sentences per 100 words
    word_list = text.split()
    intervals = range(0, len(word_list), 100)
    word_chunks = [word_list[n:n+100] for n in intervals]
    sentencesList = []
    for n in range(0,len(intervals)):
        sentences = [word.count(".") for word in word_chunks[n]]
        total = sum(sentences)
        sentencesList.extend([total])
    S = sum(sentencesList)
    if len(word_chunks) != 0:
        return float(S/len(word_chunks))
    return 0.0

def text_to_vector(text):
    WORD = re.compile(r"\w+")
    words = WORD.findall(text)
    return Counter(words)

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = sum1**0.5 * sum2**0.5

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [26]:
@udf(returnType=FloatType())
def avg_term_entropy(text):
    labels = [i for i in text.split(" ")]
    ate_val = stats.entropy(list(Counter(labels).values()), base=2)
    return ate_val.item()

@udf(returnType=FloatType())
def automated_reading_index(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_chars = character_count(text)
    if num_words != 0 and num_sentences != 0:
        return 4.71 * (character_count(text) / word_count(text) ) + 0.5 * (word_count(text) / sentence_count(text)) - 21.43
    return 0.0

@udf(returnType=FloatType())
def coleman_liau_index(text):
    L = avgLetters(text)
    S = avgSentences(text)
    return  0.588 * L - 0.296 * S  - 15.8

@udf(returnType=FloatType())
def flesch_kincaid_grade_level(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_syllables = total_syllables_count(text)
    if num_words != 0 and num_sentences != 0:
        return 0.39 * (word_count(text) / sentence_count(text)) + 11.8 * (total_syllables_count(text) / word_count(text)) - 15.9
    return 0.0

@udf(returnType=FloatType())
def flesch_reading_ease_score(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_syllables = total_syllables_count(text)
    if num_words != 0 and num_sentences != 0:    
        return 206.835 - 1.015 * (word_count(text) / sentence_count(text)) - 84.6 *(total_syllables_count(text) / word_count(text))
    return 0.0

@udf(returnType=FloatType())
def gunning_fox_index(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_diff_words = difficult_words(text)
    if num_words != 0 and num_sentences != 0:    
        return 0.4 * (word_count(text) / sentence_count(text) + 100 * (difficult_words(text)/ word_count(text)))
    return 0.0

@udf(returnType=FloatType())
def LOC(code, body):
    code_len = len(code)
    body_len = len(body)
    if code_len + body_len != 0:
        return len(code)/float(len(code)+len(body))
    return 0.0

@udf(returnType=FloatType())
def metric_entropy(text):
    return avg_term_entropy(text) / len(text)

@udf(returnType=FloatType())
def metric_entropy_temp(text):
    labels = [i for i in text.split(" ")]
    ate_val = stats.entropy(list(Counter(labels).values()), base=2)
    return (ate_val / len(text)).item()

@udf(returnType=FloatType())
def smog_index(text):
    if sentence_count(text) >= 3:
        SMOG = (1.043 * (30*(poly_syllable_count(text) / sentence_count(text)))**0.5) + 3.1291
        return SMOG
    else:
        return 0.0

@udf(returnType=IntegerType())    
def tag_count(tags):
    pattern = r"<.*?>"
    if tags is not None:
        tags_list = re.findall(pattern, tags)
        return len(tags_list)
    return 0

@udf(returnType=FloatType())
def text_similarity(text_1, text_2):
    if text_1 is None or text_2 is None:
        return 0.0
    v1 = text_to_vector(text_1)
    v2 = text_to_vector(text_2)    
    return get_cosine(v1, v2)

@udf(returnType=IntegerType())
def fetch_sentence_count(text):
    return sentence_count(text)

@udf(returnType=IntegerType())
def fetch_word_count(text):
    return word_count(text)

In [41]:
# text based features
features_df = clean_df.withColumn('ATE', avg_term_entropy(clean_df['CleanBody']))
features_df = features_df.withColumn('ARI', automated_reading_index(features_df['CleanBody']))
features_df = features_df.withColumn('CLI', coleman_liau_index(features_df['CleanBody']))
features_df = features_df.withColumn('FKGL', flesch_kincaid_grade_level(features_df['CleanBody']))
features_df = features_df.withColumn('FKES', flesch_reading_ease_score(features_df['CleanBody']))
features_df = features_df.withColumn('GFI', gunning_fox_index(features_df['CleanBody']))
features_df = features_df.withColumn('ME', metric_entropy_temp(features_df['CleanBody']))
features_df = features_df.withColumn('SI', smog_index(features_df['CleanBody']))

# code based features
features_df = features_df.withColumn('LOC', LOC(features_df['CodeBody'], features_df['CleanBody']))

# other features
features_df = features_df.withColumn('Sentences Count', fetch_sentence_count(features_df['CleanBody']))
features_df = features_df.withColumn('Word Count', fetch_word_count(features_df['CleanBody']))
features_df = features_df.withColumn('Tag Count', tag_count(features_df['Tags']))
features_df = features_df.withColumn('Title & Text Similarity', text_similarity(features_df['Title'], features_df['CleanBody']))

In [28]:
# cache the features_df for further querying
features_df.cache()

DataFrame[_c0: string, Id: string, AcceptedAnswerId: string, Score: int, ViewCount: int, Body: string, OwnerUserId: string, Title: string, Tags: string, AnswerCount: int, CommentCount: int, FavoriteCount: int, Reputation: int, Views: int, UpVotes: int, DownVotes: int, Quality: int, Text: string, CodeBody: string, CleanBody: string, ATE: float, ARI: float, CLI: float, FKGL: float, FKES: float, GFI: float, ME: float, SI: float, LOC: float, Sentences Count: int, Word Count: int, Tag Count: int, Title & Text Similarity: float]

In [29]:
features_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- AcceptedAnswerId: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- Quality: integer (nullable = true)
 |-- Text: string (nullable = true)
 |-- CodeBody: string (nullable = true)
 |-- CleanBody: string (nullable = true)
 |-- ATE: float (nullable = true)
 |-- ARI: float (nullable = true)
 |-- CLI: float (nullable = true)
 |-- FKGL: float (nullable = true)
 |-- FKES: float (nullable = true)
 |-- GFI: float (nu

In [30]:
features_df.columns

['_c0',
 'Id',
 'AcceptedAnswerId',
 'Score',
 'ViewCount',
 'Body',
 'OwnerUserId',
 'Title',
 'Tags',
 'AnswerCount',
 'CommentCount',
 'FavoriteCount',
 'Reputation',
 'Views',
 'UpVotes',
 'DownVotes',
 'Quality',
 'Text',
 'CodeBody',
 'CleanBody',
 'ATE',
 'ARI',
 'CLI',
 'FKGL',
 'FKES',
 'GFI',
 'ME',
 'SI',
 'LOC',
 'Sentences Count',
 'Word Count',
 'Tag Count',
 'Title & Text Similarity']

In [42]:
pd.DataFrame(features_df.take(50), columns=features_df.columns)

,_c0,Id,AcceptedAnswerId,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,Reputation,Views,UpVotes,DownVotes,Quality,Text,CodeBody,CleanBody,ATE,ARI,CLI,FKGL,FKES,GFI,ME,SI,LOC,Sentences Count,Word Count,Tag Count,Title & Text Similarity
0,0,8019330,8019355.0,-1,246,<p>My lines:</p>\n\n<pre><code>ABC: XYZ\nXYZ A...,935420,"In Perl, how can I join an input line with the...",<perl>,6,4,0,103,31,0,0,0,my lines:\nabc: xyz\nxyz abc: xyz\nxyz:\nabc\n...,ABC: XYZ\nXYZ ABC: XYZ\nXYZ:\nABC\nXYZ\nABC:\n...,My lines:\n\n\n\nI want to join lines end with...,3.418296,0.000000,13.600000,0.000000,0.000000,0.000000,0.050269,0.0000,0.585366,0,0,1,0.412514
1,5,13579810,13579922.0,54,259088,<p>I have a 350MB file named <code>text_file.t...,1342109,How to import data from text file to mysql dat...,<mysql><import><text-files>,8,0,32,19464,1669,83,1,1,i have a 350mb file named text_file.txt contai...,text_file.txt345868230 1646198120 153128314...,I have a 350MB file named containing this tab...,5.389433,25.090073,141.483994,20.392662,98.473991,34.099998,0.015759,3.1291,0.375912,4,341,3,0.396942
2,8,3934106,3934115.0,10,3153,<p>This is a curiosity more than anything: Doe...,348056,Unicode based programming language,<unicode><programming-languages>,13,10,5,20747,1040,461,17,1,this is a curiosity more than anything: does t...,,This is a curiosity more than anything: Does t...,6.238904,53.585064,156.776001,42.810883,39.180481,56.880001,0.008763,3.1291,0.000000,5,711,2,0.251976
3,9,4724606,4724676.0,60,56061,<p>I need to check (in Javascript) if a CSS fi...,244415,How to use Javascript to check and load CSS if...,<javascript><jquery><css>,14,0,19,3631,80,573,1,1,i need to check (in javascript) if a css file ...,,I need to check (in Javascript) if a CSS file ...,4.186704,7.078485,29.472000,6.503990,134.374313,19.799999,0.041867,0.0000,0.000000,2,99,3,0.656599
4,12,28980152,31768957.0,12,17287,<p>Is there a way to play sound out of the App...,2158235,Play sound from Apple Watch speaker,<ios><objective-c><iphone><audio><apple-watch>,6,6,8,379,46,40,1,1,is there a way to play sound out of the apple ...,,Is there a way to play sound out of the Apple ...,4.368523,11.210870,39.175999,9.705870,125.667282,23.000000,0.037660,0.0000,0.000000,2,115,5,0.326599
5,15,307688,None,376,507072,<p>What is a simple way of downloading a file ...,20837,How to download a file from a URL in C#?,<c#><downloadfile>,11,1,70,4970,226,5,1,1,what is a simple way of downloading a file fro...,,What is a simple way of downloading a file fro...,3.334679,11.901864,11.836000,10.110000,125.441528,23.600000,0.055578,0.0000,0.000000,1,59,2,0.596040
6,20,913671,914383.0,32,25655,<p>Are there native code compilers for Lisp? T...,45269,Are there Lisp native code compilers?,<compiler-construction><lisp>,6,4,30,5625,411,55,3,1,are there native code compilers for lisp? to w...,,Are there native code compilers for Lisp? To w...,4.700440,21.045975,60.051998,17.348183,105.607269,30.799999,0.030325,0.0000,0.000000,2,154,2,0.454859
7,24,1203892,1205824.0,215,174146,<p>I need to know the current route in a filte...,120496,How can I find out the current route in Rails?,<ruby-on-rails><ruby><routing>,13,2,69,10691,426,86,9,1,i need to know the current route in a filter i...,,I need to know the current route in a filter i...,4.664498,5.219465,47.112000,4.924599,138.870193,18.266666,0.033801,3.1291,0.000000,3,137,3,0.666886
8,27,15434728,15434783.0,27,72739,"<p>I have a file containing many lines, and I ...",1003575,How to display the first word of each line in ...,<linux><shell><grep>,6,3,11,34364,4157,1079,35,1,"i have a file containing many lines, and i wan...",,"I have a file containing many lines, and I wan...",4.504706,16.414190,47.408001,13.743529,115.420883,27.200001,0.032881,0.0000,0.000000,2,136,3,0.549972
9,28,6347039,6347078.0,9,17203,"<p>I'm developing a web application, using ASP...",586528,How to process payments through cards?,<c#><asp.net><e-commerce><paymen

In [58]:
print("Features Dataset size: ", features_df.count())
print("Features Dataset size with quality 0: ", features_df.select('*').where(features_df['Quality'] == 0).distinct().count())
print("Features Dataset size with quality 1: ", features_df.select('*').where(features_df['Quality'] == 1).count())

Features Dataset size:  79246


Features Dataset size with quality 0:  6702


Features Dataset size with quality 1:  23312


### Summary of low quality data

In [56]:
low_quality = features_df.select('*').where(col('Quality') == 0)
lq_desc = low_quality.describe('*')
lq_desc.show()


+-------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+--------------------+-----------------+-------------------+-----------------+------------------+-------------------+-----------------------+
|summary|                 _c0|                  Id|    AcceptedAnswerId|             Score|         ViewCount|                Body|         OwnerUserId|               Title|                Tags|        AnswerCount|      CommentCount|     FavoriteCount|       Reputation|             Views|           UpVotes|         DownVotes|Quality|                

### Summary of high quality data

In [45]:
high_quality = features_df.select('*').where(features_df['Quality'] == 1).sample(fraction=0.3)
hq_desc = high_quality.describe('*')
pd.DataFrame(hq_desc.take(50), columns=hq_desc.columns)


,summary,_c0,Id,AcceptedAnswerId,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,Reputation,Views,UpVotes,DownVotes,Quality,Text,CodeBody,CleanBody,ATE,ARI,CLI,FKGL,FKES,GFI,ME,SI,LOC,Sentences Count,Word Count,Tag Count,Title & Text Similarity
0,count,6770,6770,121,6770,6770,6770,6770,346,345,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770,6770
1,mean,24470.003692762188,2.7545052352732643E7,1.002132379338843E7,3.3373707533234858,1288.962186115214,None,2488885.1367799114,None,None,0.172821270310192,1.1838995568685378,0.29320531757754803,78367.03515509602,16304.409453471197,2130.370753323486,1474.9320531757755,1.0,None,5.444444443333334,None,4.447886878119966,19.948848511949677,83.19029041599356,16.17074748077175,78.97230285849847,26.843044972498976,0.03068345915845563,1.3607194458188123,0.32538068102551637,3.040472673559823,228.25805022156572,0.1432791728212703,0.014201803842182526
2,stddev,17938.899528341084,2.057991875207421E7,1.277559038802671E7,19.455353411406918,16825.303883060245,None,3250437.296835146,None,None,1.2354515529857952,2.012886724026233,4.127545972828031,192903.5405259784,78615.84857661164,4787.43404652241,6927.1196278194575,0.0,None,4.857220667174761,None,1.3329304543242169,23.00357661532518,69.70789221073139,18.13825346304135,57.473566938794285,21.326231865423026,0.02267782176965726,1.5513302054073572,0.3236427131502105,9.871376302072878,330.82149105395507,0.6733448546638751,0.0725784571942848
3,min,100,10000351,10046223.0,-16,0,"""The web storage is a standardised way of prov...",1000485,"""Replication status """"Fails"""" when used the na...",<.net><angular><session><asp.net-web-api>,0,0,0,1,0,0,0,1,\n\n\n program = test-program\n cflags =...,,\n,0.0,-14.9265,-15.8,-12.376667,-412.9759,0.0,0.0,0.0,0.0,0,0,0,0.0
4,max,9999,9991191,9942747.0,612,507072,Wing IDE is an integrated development environm...,999042,why is this error being thrown from my web config,<wordpress><woocommerce><hook-woocommerce>,23,24,170,1225022,2000044,57879,78560,1,{{ content }} works and it's different than {{...,☼sedecho $myString | sed 's/☼/ /g'\n,"zero sized malloc, calloc is 100% OK. \n is OK...",7.6303577,276.89236,617.1997,216.8752,190.48,235.6,0.2,3.1291,0.99950886,768,16614,5,1.0


## Convert to CSV

In [49]:
# write in only one csv
STORAGE_LOC = "gs://eecs-e6893-project-42/dataset/clean_data_80k.csv"

In [50]:
features_df.coalesce(1).write.format("csv").option("header", "true").save(STORAGE_LOC)